In [5]:
#concat相对于merge，是一种比较简单的合并
#maerge在考虑索引或者key的对照下，实现DataFrame的合并
import pandas as pd
import numpy as np

#merge two df by key/keys（maybe used in database）
#simple example
left=pd.DataFrame({'key':['K0','K1','K2','K3'],
                  'A':['A0','A1','A2','A3'],
                  'B':['B0','B1','B2','B3']})
right=pd.DataFrame({'key':['K0','K1','K2','K3'],
                  'C':['C0','C1','C2','C3'],
                  'D':['D0','D1','D2','D3']})
print(left)
print(right)
res=pd.merge(left,right,on='key') #on='key'基于key列合并
print(res)

  key   A   B
0  K0  A0  B0
1  K1  A1  B1
2  K2  A2  B2
3  K3  A3  B3
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3
  key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3


In [10]:
#merge two df by key/keys (maybe used in database)
#consider two keys
left=pd.DataFrame({'key1':['K0','K0','K1','K2'],
                   'key2':['K0','K1','K0','K1'],
                  'A':['A0','A1','A2','A3'],
                  'B':['B0','B1','B2','B3']})
right=pd.DataFrame({'key1':['K0','K1','K1','K2'],
                    'key2':['K0','K0','K0','K0'],
                  'C':['C0','C1','C2','C3'],
                  'D':['D0','D1','D2','D3']})
print(left)
print(right)
#how=['left','right','inner','outer']
res1=pd.merge(left,right,on=['key1','key2']) #how默认为inner，合并后保留相同的部分
#在left中‘K1,K0’只有一组对应‘A2,B2’，但right有两组‘K1,K0’,所以‘A2,B2’出现了两遍
res2=pd.merge(left,right,on=['key1','key2'],how='right') #基于right的keys合并（即right的keys全保留）
print(res2)

  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
  key1 key2   C   D
0   K0   K0  C0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3
  key1 key2    A    B   C   D
0   K0   K0   A0   B0  C0  D0
1   K1   K0   A2   B2  C1  D1
2   K1   K0   A2   B2  C2  D2
3   K2   K0  NaN  NaN  C3  D3


In [13]:
#merge two df by key/keys (maybe used in database)
#indicator
df1=pd.DataFrame({'col1':[0,1],'col_left':['a','b']})
df2=pd.DataFrame({'col1':[1,2,2],'col_right':[2,2,2]})
print(df1)
print(df2)
res=pd.merge(df1,df2,on='col1',how='outer',indicator=True) 
#indicator会显示每行merge的方式，indicator默认为False
print(res)
#显示结果多了label是‘merge’的一列，其中为left_only表示只有左边有数据，右边没有数据
#merge（x,y）,x是左边，y是右边

#将indicator默认的列名‘merge’，改为自定义的
res1=pd.merge(df1,df2,on='col1',how='outer',indicator='indicator_column') 
print(res1)

   col1 col_left
0     0        a
1     1        b
   col1  col_right
0     1          2
1     2          2
2     2          2
   col1 col_left  col_right      _merge
0     0        a        NaN   left_only
1     1        b        2.0        both
2     2      NaN        2.0  right_only
3     2      NaN        2.0  right_only
   col1 col_left  col_right indicator_column
0     0        a        NaN        left_only
1     1        b        2.0             both
2     2      NaN        2.0       right_only
3     2      NaN        2.0       right_only


In [20]:
#merge by index
left=pd.DataFrame({'A':['A0','A1','A2'],
                  'B':['B0','B1','B2']},
                  index=['K0','K1','K2'])
right=pd.DataFrame({ 'C':['C0','C1','C2'],
                  'D':['D0','D1','D2']},
                  index=['K0','K2','K3'])
print(left)
print(right)

#left_index和right_index,默认值都是False
#当把他们都设为True时，表示不再根据列中的‘key’进行合并，而是调转方向，根据行名来merge
res1=pd.merge(left,right,left_index=True,right_index=True,how='outer')
print(res1)


     A   B
K0  A0  B0
K1  A1  B1
K2  A2  B2
     C   D
K0  C0  D0
K2  C1  D1
K3  C2  D2
      A    B    C    D
K0   A0   B0   C0   D0
K1   A1   B1  NaN  NaN
K2   A2   B2   C1   D1
K3  NaN  NaN   C2   D2


In [24]:
#处理overlapping
boys=pd.DataFrame({'k':['K0','K1','K2'],'age':[1,2,3]})
girls=pd.DataFrame({'k':['K0','K0','K3'],'age':[4,5,6]})
print(boys)
print(girls)
#'age'该列名重复
res=pd.merge(boys,girls,on='k',suffixes=['_boy','_girl'],how='outer')
#suffixes修改列名（加后缀）
print(res)

    k  age
0  K0    1
1  K1    2
2  K2    3
    k  age
0  K0    4
1  K0    5
2  K3    6
    k  age_boy  age_girl
0  K0      1.0       4.0
1  K0      1.0       5.0
2  K1      2.0       NaN
3  K2      3.0       NaN
4  K3      NaN       6.0
